In [1]:
import sys
sys.path.append('/home/nick/python/asop_global/ASoP-Coherence')
import asop_coherence as asop
import iris
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from numba import jit
from tqdm import tqdm
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units

In [2]:
def interp_3x3(cube):
    print('Interpolating to 3x3 ...')
    print(cube)
    lonin = cube.coord('longitude').points
    latin = cube.coord('latitude').points
    interp_lon = iris.coords.DimCoord(np.arange(min(lonin),max(lonin),3),standard_name='longitude',units='degrees_east',circular=True)
    nlon = len(interp_lon.points)
    interp_lat = iris.coords.DimCoord(np.arange(min(latin),max(latin),3),standard_name='latitude',units='degrees')
    nlat = len(interp_lat.points)
    interp_cube = iris.cube.Cube(data=np.empty((nlat,nlon)),dim_coords_and_dims=[(interp_lat,0),(interp_lon,1)])
    interp_cube.coord('longitude').guess_bounds()
    interp_cube.coord('latitude').guess_bounds()
    out_cube = cube.regrid(interp_cube,iris.analysis.AreaWeighted(mdtol=0))
    print(out_cube)
    return(out_cube)

In [3]:
def load_cmip6(asop_dict,overwrite_interp=False):
    from iris.util import unify_time_units
    from iris.experimental.equalise_cubes import equalise_attributes
    from iris.time import PartialDateTime

    constraint = iris.Constraint(time = lambda cell: PartialDateTime(year=asop_dict['start_year']) <= cell <= PartialDateTime(year=asop_dict['stop_year']),latitude = lambda cell: -60 <= cell <= 60)
    cubelist = iris.load(str(asop_dict['dir'])+'/pr_3hr*.3x3.nc') # Use NetCDF3 data to save compute time
    unify_time_units(cubelist)
    equalise_attributes(cubelist)
    cube = cubelist.concatenate_cube()
    cube.coord('time').bounds = None
    return(cube)

In [4]:
def get_asop_dict(key):
    datapath=Path('/media/nick/lacie_tb3/data_from_gill/CMIP6')
    if key == 'AWI':
        asop_dict={
            'desc': 'AWI-CM-1-1-MR_historical_r1i1p1f1_gn',
            'dir': datapath/'AWI-CM-1-1-MR',
            'name': 'AWI',
            'start_year': 1990,
            'stop_year': 2014,
            'dt': 10800,
            'legend_name': 'AWI',
            'region': [-90,90,0,360],
            'color': 'red',
            'symbol': '<'
        }
    elif key == 'BCC':
        asop_dict={
            'dir': datapath/'BCC-CSM2-MR',
            'name': 'BCC',
            'dt': 10800,
            'legend_name': 'BCC',
            'region': [-90,90,0,360],
            'color': 'blue',
            'symbol': '8'
        }
    else:
        raise Exception('No dictionary for '+key)
    return(asop_dict)

In [5]:
regions = [ ([-30,30,0,360],'land','trop_land'),
            ([-30,30,0,360],'ocean','trop_ocean'),
            ([-90,-30,0,360],'land','sh_land'),
            ([-90,-30,0,360],'ocean','sh_ocean'),
            ([30,90,0,360],'land','nh_land'),
            ([30,90,0,360],'ocean','nh_ocean'),
            ([-90,90,0,360],'land','glob_land'),
            ([-90,90,0,360],'ocean','glob_ocean')]
datasets=['AWI'] #,'BCC']
n_datasets=len(datasets)
n_regions = len(regions)
space_metrics_plot = np.empty((n_datasets,n_regions))
time_metrics_plot = np.empty((n_datasets,n_regions))
all_datasets = []
all_colors = []
all_symbols = []
all_regions = []
for box,mask_type,region_name in regions:
	all_regions.append(region_name)

In [6]:
def compute_clim_contribution(precip,overwrite=False):
    import iris.coord_categorisation

    clim_contrib_filename='pr_clim_month_frac.'+str(asop_dict['start_year'])+'-'+str(asop_dict['stop_year'])+'.3x3.nc'
    clim_contrib_file=asop_dict['dir']/clim_contrib_filename
    if clim_contrib_file.exists() and not overwrite:
        month_frac = iris.load_cube(str(clim_contrib_file))
    else:
        if not 'month_number' in [coord.name() for coord in precip.coords()]:
            iris.coord_categorisation.add_month_number(precip,'time')
        month_clim = precip.aggregated_by('month_number',iris.analysis.SUM)
        ann_clim = month_clim.collapsed('time',iris.analysis.SUM)
        month_frac = month_clim/ann_clim
        month_frac.var_name='clim_precip_frac'
        month_frac.long_name='Fractional monthly precipitation contribution to annual precipitation'
        iris.save(month_frac,str(clim_contrib_file))
    return(month_frac)

In [7]:
#for model in datasets:
model = 'AWI'
asop_dict = get_asop_dict(model)
precip = load_cmip6(asop_dict,overwrite_interp=True)
#precip.convert_units('mm day-1')

In [8]:
clim_contrib = compute_clim_contribution(precip,overwrite=True)

In [9]:
def new_cube_copy(cube,var_name,long_name):
    new_cube = cube.copy()
    new_cube.var_name=var_name
    new_cube.long_name=long_name
    return(new_cube)

In [10]:
def compute_temporal_summary(precip,clim_contrib,ndivs,twod=False,min_precip_threshold=1/86400.0,wet_season_threshold=1.0/24.0):
    import numpy.ma as ma

    # Compute temporal summary metric only
    if not 'month_number' in [coord.name() for coord in precip.coords()]:
        iris.coord_categorisation.add_month_number(precip,'time')
    lon_coord = precip.coord('longitude')
    lat_coord = precip.coord('latitude')
    nlon = len(lon_coord.points)
    nlat = len(lat_coord.points)
    #print(precip.coord('month_number').points)
    month_coord = iris.coords.DimCoord(clim_contrib.coord('month_number').points,var_name='month_number')
    #month_coord =clim_contrib.coord('month_number')
    nmonths = len(month_coord.points)

    lower_thresh = iris.cube.Cube(data=np.empty((nmonths,nlat,nlon)),dim_coords_and_dims=[(month_coord,0),(lat_coord,1),(lon_coord,2)])
    lower_thresh.var_name='lower_threshold'
    lower_thresh.long_name='Lower (off) threshold based on '+str(ndivs)+' divisions'
    upper_thresh = lower_thresh.copy()
    upper_thresh.var_name='upper_threshold'
    upper_thresh.long_name='Upper (on) threshold based on '+str(ndivs)+' divisions'
    if twod:
        time_inter = new_cube_copy(lower_thresh,'temporal_onoff_metric','Temporal intermittency on-off metric based on '+str(ndivs)+' divisions')
        onon_freq = new_cube_copy(lower_thresh,'prob_onon','Probability of upper division followed by upper division')
        onoff_freq = new_cube_copy(lower_thresh,'prob_onoff','Probability of upper division followed by lower division')
        offon_freq = new_cube_copy(lower_thresh,'prob_offon','Probability of lower division followed by upper division')
        offoff_freq = new_cube_copy(lower_thresh,'prob_offoff','Probability of lower division followed by lower division')

        #time_inter_mean = iris.cube.Cube(data=np.empty((nlat,nlon)),dim_coords_and_dims=[(lat_coord,1),(lon_coord,2)])
        #time_inter.var_Name='temporal_onoff_metric_mean'
        #time_inter.long_name='Temporal intermittency on-off metric based on '+str(ndivs)+' divisions (mean of all months in wet season)'
        #onon_freq_mean = new_cube_copy(time_inter_mean,'prob_onon_mean','Probability of upper division followed by upper division (mean of all months in wet season)')
        #onoff_freq_mean = new_cub

    # Use cube slices to avoid loading all data into memory
    for t,t_slice in enumerate(precip.slices(['time'])):
        lat = t // nlon
        lon = t % nlon
        month_list = []
        for m,month in enumerate(clim_contrib.coord('month_number').points):
            if clim_contrib.data[m,lat,lon] > wet_season_threshold:
                month_constraint = iris.Constraint(month_number=month)
                this_month = t_slice.extract(month_constraint)
            #wet_season = find_wet_season(t_slice,clim_contrib[:,lat,lon])
                this_precip = this_month.data[np.where(this_month.data > min_precip_threshold)]
                nt = np.size(this_precip)
                if nt > ndivs:
                    lower_thresh.data[m,lat,lon],upper_thresh.data[m,lat,lon],offoff_freq.data[m,lat,lon],offon_freq.data[m,lat,lon],onoff_freq.data[m,lat,lon],onon_freq.data[m,lat,lon] = compute_onoff_metric(this_precip.data,this_precip.data)
                    print(lat,lon,m,onon_freq.data[m,lat,lon])
                else:
                    lower_thresh.data[m,lat,lon] = ma.masked
                    upper_thresh.data[m,lat,lon] = ma.masked
                    offoff_freq.data[m,lat,lon] = ma.masked
                    offon_freq.data[m,lat,lon] = ma.masked
                    onoff_freq.data[m,lat,lon] = ma.masked
                    onon_freq.data[m,lat,lon] = ma.masked
            else:
                lower_thresh.data[m,lat,lon] = ma.masked
                upper_thresh.data[m,lat,lon] = ma.masked
                offoff_freq.data[m,lat,lon] = ma.masked
                offon_freq.data[m,lat,lon] = ma.masked
                onoff_freq.data[m,lat,lon] = ma.masked
                onon_freq.data[m,lat,lon] = ma.masked

    time_inter.data = 0.5*((onon_freq.data+offoff_freq.data)-(onoff_freq.data+offon_freq.data))
    time_inter_mean = time_inter.collapsed('month_number',iris.analysis.MEAN,mdtol=0)  
    time_inter_mean.data = np.nanmean(time_inter.data,axis=0)
    time_inter_mean.var_name='temporal_onoff_metric_mean'
    time_inter_mean.long_name='Temporal intermittency on-off metric based on '+str(ndivs)+' divisions (mean of all months in wet season)'
    onon_freq_mean = onon_freq.collapsed('month_number',iris.analysis.MEAN)
    onon_freq_mean.data = np.nanmean(onon_freq.data,axis=0)
    onon_freq_mean.var_name='prob_onon_mean'
    onon_freq_mean.long_name='Probability of upper division followed by upper division (mean of all months in wet season)'
    onoff_freq_mean = onoff_freq.collapsed('month_number',iris.analysis.MEAN)
    onoff_freq_mean.data = np.nanmean(onoff_freq.data,axis=0)
    onoff_freq_mean.var_name='prob_onoff_mean'
    onoff_freq_mean.long_name='Probability of upper division followed by lower division (mean of all months in wet season)'
    offon_freq_mean = offon_freq.collapsed('month_number',iris.analysis.MEAN)
    offon_freq_mean.data = np.nanmean(offon_freq.data,axis=0)
    offon_freq_mean.var_name='prob_offon_mean'
    offon_freq_mean.long_name='Probability of lower division followed by upper division (mean of all months in wet season)'
    offoff_freq_mean = offoff_freq.collapsed('month_number',iris.analysis.MEAN)
    offoff_freq_mean.data = np.nanmean(offoff_freq.data,axis=0)
    offoff_freq_mean.var_name='prob_offoff_mean'
    offoff_freq_mean.long_name='Probability of lower division followed by lower division (mean of all months in wet season)'
    out_cubelist = [time_inter,onon_freq,onoff_freq,offon_freq,offoff_freq,lower_thresh,upper_thresh,time_inter_mean,onon_freq_mean,onoff_freq_mean,offon_freq_mean,offoff_freq_mean]
    return(out_cubelist)

In [11]:
@jit(nopython=True,parallel=True)
def compute_onoff_metric(wet_season,this_precip):
    lower_thresh = np.percentile(this_precip,25)
    upper_thresh = np.percentile(this_precip,75)
    upper_mask = np.where(wet_season >= upper_thresh,1,0)
    lower_mask = np.where(wet_season <= lower_thresh,1,0)
    non = np.sum(upper_mask)
    noff = np.sum(lower_mask)
    onon = upper_mask + np.roll(upper_mask,1)
    onon_count = np.count_nonzero(np.where(onon == 2,1,0))
    onoff = upper_mask + np.roll(lower_mask,1)
    onoff_count = np.count_nonzero(np.where(onoff == 2,1,0))
    offon = lower_mask + np.roll(upper_mask,1)
    offon_count = np.count_nonzero(np.where(offon == 2,1,0))
    offoff = lower_mask + np.roll(lower_mask,1)
    offoff_count = np.count_nonzero(np.where(offoff == 2,1,0))
    onon_freq = onon_count/float(non)
    onoff_freq = onoff_count/float(non)
    offon_freq = offon_count/float(noff)
    offoff_freq = offoff_count/float(noff)

    return(lower_thresh,upper_thresh,offoff_freq,offon_freq,onoff_freq,onon_freq)

In [12]:
print(precip)
print(clim_contrib)
time_inter = compute_temporal_summary(precip,clim_contrib,4,twod=True)
iris.save(time_inter,'asop_coherence_global_cmip6_timeinter.nc')

 0.7083333333333334
4 5 5 0.7407407407407407
4 5 6 0.6538461538461539
4 5 7 0.6896551724137931
4 5 8 0.75
4 5 9 0.5641025641025641
4 5 10 0.5666666666666667
4 5 11 0.5714285714285714
4 6 0 0.6111111111111112
4 6 1 0.5333333333333333
4 6 2 0.6071428571428571
4 6 3 0.675
4 6 4 0.6486486486486487
4 6 5 0.6206896551724138
4 6 6 0.6071428571428571
4 6 7 0.5172413793103449
4 6 8 0.6451612903225806
4 6 9 0.5806451612903226
4 6 10 0.52
4 6 11 0.4666666666666667
4 7 0 0.6086956521739131
4 7 1 0.6428571428571429
4 7 2 0.5925925925925926
4 7 3 0.6153846153846154
4 7 4 0.6842105263157895
4 7 5 0.6206896551724138
4 7 6 0.76
4 7 7 0.5454545454545454
4 7 8 0.75
4 7 9 0.6071428571428571
4 7 10 0.52
4 7 11 0.5
4 8 0 0.5555555555555556
4 8 1 0.5384615384615384
4 8 2 0.6666666666666666
4 8 3 0.6388888888888888
4 8 4 0.6333333333333333
4 8 5 0.5555555555555556
4 8 6 0.6956521739130435
4 8 7 0.5
4 8 8 0.6363636363636364
4 8 9 0.6
4 8 10 0.5
4 8 11 0.46153846153846156
4 9 0 0.4444444444444444
4 9 1 0.583333

KeyboardInterrupt: 